<a href="https://colab.research.google.com/github/rasthh/regnlydenud/blob/master/RegnLydenUd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Regn lyden ud skitse](https://www.dtu.dk/-/media/DTUdk/Forskning/ForskningensDoegn/2019/Regn-lyden-ud/Lyden_940.ashx?h=370&la=da&mw=940&w=940&hash=DA751E68D0A1608061295B8380528638F97652C8)

#Regn lyden ud (Lyden af Danmark)

![DTU logo](https://www.dtu.dk/-/media/DTU_Generelt/Navnetraek/Nyt-design/Corp_Red_RGB-web1.ashx?mh=100&mw=460&hash=1E36B0CED9104CC33A07599A5A4653E0A9A8DCD5 =50x)
![Forsknings døgn](https://pbs.twimg.com/profile_images/3184167407/9680b7b1fd0399b0aaaf331490e6d4cd_400x400.jpeg =100x)

Du skal nu til at kigge på lyd, og se på lyden på samme måde, som kunstig intelligens kigger på lyd. Ved at kigge på forskellige lyde - og deres spectrogrammer - skal du lære at afkode, hvilken slags lyd som gemmer sig bag spectrogrammerne.

#### Moderne notesbog
Det du har åbnet her er en moderne notesbog. Forskellen mellem en gammeldags notesbog og denne er, at der hører en computer til denne notesbog. Den står et helt andet sted i et server-rum, og vi kan bruge den til at beregne ting, alt imens du læser notesbogen. Tag et kig oppe i højre hjørne  (hvis ikke det står der endnu, så kommer det når du læser lidt videre) -  der skulle du gerne kunne se et grønt hak, som viser, at du er korrekt koblet op til computeren, samt hvor meget af computerens RAM og disk plads du bruger lige nu.

#### Optaget lyd
Det du skal bruge notesbogen til er at kigge på de lyde, som du har optaget med Lyden af Danmark-appen. Inden du går i gang med at arbejde med denne notesbog skulle du gerne have optaget en masse lyde, og noteret deres tildelte "nummer" i Lyden af Danmark-databsen (f.eks. #1561). Hvis du downloadede filerne undervejs, og har dem gemt på din computer, så er du klar til at gå videre. Hvis ikke du har filerne på din egen computer endnu, så gå ind på [Lyden af Danmark-hjemmesiden](https://lydenafdk.prod.b14cms.dk/), og download dine lyde til den computer du sidder ved - læg dem et sted, som du kan huske hvor er. Sørg også for at give filnavne et simpelt navn, som forklarer, hvad du har optaget i den fil (f.eks. "fuglesang.mp3"). Undgå mellemrum og punktummer i dine filnavne.

#### Opsætning
Før du for alvor går i gang, så skal du installere et par ting på den computer, som du har fået tildelt til din notesbog. Det foregår ganske simpelt ved, at du trykker på play knappen til venstre for boksen herunder (den kommer frem når du rykker musen ind mellem de kantede parenteser). Når koden er kørt, skriver den "Opsætning færdig", og play knappen bliver erstattet med et tal (tallet indikerer rækkefølgen af "bokse" du har kørt/trykket play på).


In [0]:
#@title Kør opsætning (OBS! Kør inden du går videre, tryk til venstre i kantede parenteser)
!pip install pydub

from google.colab import files
import matplotlib.pyplot as plt
from pydub import AudioSegment
from pydub.playback import play
from IPython.display import Audio
from IPython.display import display
from IPython.display import clear_output
import numpy as np
from matplotlib.colors import ListedColormap
from scipy import signal

import numpy as np
import os

from scipy.signal import decimate

clear_output()

print('Opsætning færdig.')


## Upload 
Du har to muligheder for at bruge dine optagelser i notesbogen, og du skal kun gøre én af dem.
Løsning (*Drive*) kan være hurtigere, og kan deles på tværs af flere mennesker nemt.

*   (*Direkte*) Upload filerne direkte fra din egen computer. Kør kun *Direkte* herunder.
  *   Når du trykker på play-knappen på den *Direkte* boksen, kommer der en knap op, som giver dig mulighed for at uploade dine optagede lyde. Ved at trykke på "Browser...", og navigere til alle dine optagede lyde (som er f.eks. MP3-filer), kan du uploade lyden til computeren, som er koblet til notesbogen.
*   (*Drive*) Upload filerne til Google Drive, og hent dem derefter gennem Drive. Kør kun *Drive* herunder.
  * Inden du kører *Drive* boksen skal du skrive, hvad du har kaldt din mappe med optagelse på Google Drive (f.eks. regnlydenud-optagelser). Når du trykker play på *Drive* boksen, skal du trykke på det link der kommer frem (åbnes i ny side), logge in, kopiere koden, og sætte den ind i feltet der kommer frem på denne side. 


In [0]:
#@title (*Direkte*) Upload filer
uploaded = files.upload()
x = dict()
for fn in uploaded.keys():
  name_str = str(fn).split('.')[0]
  format_str = str(fn).split('.')[1]
  sound=AudioSegment.from_file(fn, format_str)
  x[name_str] = sound.get_array_of_samples()  
  
fs = sound.frame_rate
clear_output()

print('Upload færdig.')

In [0]:
#@title (*Drive*) Hent fra Drive
from google.colab import drive
drive.mount('/content/gdrive')
print('')

placering = "regnlydenud-test" #@param {type:"string"}
root_path = 'gdrive/My Drive/' + placering + '/'
content = os.listdir(root_path) 
print('Filer i den angivne mappe:')
print('')
for e in content:
  print(e)

print('')
print('Henter filerne og gemmer i Colab variabel...')
x = dict()
for fn in content: #uploaded.keys():
  name_str = str(fn).split('.')[0]
  #print('Behandler %s ...' % name_str)
  format_str = str(fn).split('.')[1]
  sound=AudioSegment.from_file(root_path + fn, format_str)
  x[name_str] = sound.get_array_of_samples()  
  
fs = sound.frame_rate

#clear_output()

print('')
print('Filer hentet fra Drive.')



## Lyt og visualisér
Vi kan lytte til de filer, som notesbogen nu kan tilgå. Vi kan også visualisere lyden på forskellige måder. Én måde er at vise amplituden af lydbølgen/bølgeformen som funktion af tid, og en anden er at vise spektrogrammet.

Tryk play herunder for lytte til optagelse, samt at se de to visualiseringsmetoder for en af de uploadede lyde (skriv det navn I har givet filen for at vælge hvilken I vil se). Læg mærke til, hvordan de figurer kan sammenlignes, da de har den samme førsteakse (tid). 

Farven på spektrogrammet kan tolkes som mængden af energi ved en given frekvens. Der hvor der er mindst energi er billedet blåt, og der hvor der er mest energi er billedet gult.

In [0]:
#@title Bølgeform og spektrogram
fil_til_visning = "fugle" #@param {type:"string"}
k = fil_til_visning
v = x[k]
title_str = k

print('Afspilning af: %s'%k)
display(Audio(v,rate=fs))
print('')

f, axs = plt.subplots(2, 2, gridspec_kw = {'width_ratios':[20,1]})
waveform = axs[0,0]
spectrogram = axs[1,0]
cbar = axs[1,1]
axs[0, 1].axis('off')

plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=.5)


q = 10
v_down = decimate(v, q)
fs_down = fs/q
v_down = decimate(v_down, q)
fs_down = fs_down/q
t = np.linspace(0,(len(v_down)-1)/fs_down, len(v_down))

waveform.plot(t, v_down)
waveform.set_title('Bølgeform: '+title_str)
waveform.set_xlabel('Tid [s]')
waveform.set_ylabel('Amplitude')
waveform.set_xlim([0, np.max(t)])

f, t, Zxx = signal.stft(v,fs=fs, nperseg=2**12)
idx = (100 < f) & (f < 10000)
Zxx_dB = 10*np.log10(np.abs(Zxx[idx, :]))
im = spectrogram.pcolormesh(t,np.log10(f[idx]), Zxx_dB,
                 cmap = plt.get_cmap('viridis'),
                           vmin=-20, vmax=np.max(Zxx_dB))
locs = np.log10([100, 250, 500, 1000, 2500, 5000, 10000])
spectrogram.set_yticks(locs)
spectrogram.set_yticklabels([str(int(round(10**e,0))) for e in locs])
spectrogram.set_title('Spektrogram: '+ title_str)
spectrogram.set_xlabel('Tid [s]')
spectrogram.set_ylabel('Frekvens [Hz]')
spectrogram.set_xlim([0, np.max(t)])
plt.colorbar(im, cax=cbar)
cbar.set_ylabel('STFT magnitude [dB]', labelpad=10, rotation=270)


plt.show()  

## Spectrogrammer
For at kunne sammenligne alle lydenes spektrogrammer, viser vi dem nu sammen i et stort plot - dog uden akser og tilsvarende.

Start med at kigge på spektrogrammerne ved at trykke play på boksen "beregn og visualisér spektrogrammer". I kan få figuren til at fylde hele skærmen ved at trykke på boksen, for derefter at trykke på de 3 prikker i højre hjørne af boksen, og derefter "View output fullscreen" (når figuren er lavet).

De titler I havde givet jeres lydfiler fremgår i rød tekst. Prøv og se om I kan se forskel på de forskelle lyde. Er der nogen der ligner hinanden? 

Husk, at første aksen (fra venstre mod højre) er tid, og anden aksen (fra ned mod op) er frekvens. Hvis der er gule/grønne områder "oppe" i billede så betyder det højfrekvente lyde, mens "start" og "stop" af gule/grønne områder langs førsteaksen betyder pauser i tid. I har måske set et gammeldags fjernsyn med et billede af støj (sort/hvid-flimmer) - minder nogle af lydene om dét (bare i blå/grøn/gul)?

I kan ændre størrelsen på figuren ved at rykke på slideren "stoerrelse".

In [0]:
#@title Beregn og visualiser spektrogrammer

stoerrelse = 1 #@param {type:"slider", min:0.1, max:2, step:0.01}

n = len(x.keys())
k2 = int(3)
k1 = int(np.ceil(n/k2))
figsize = (stoerrelse*k2*5,stoerrelse*k1*3)
fig, _axs = plt.subplots(nrows=k1, ncols=k2, figsize=figsize)
axs = _axs.flatten()
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, 
                    wspace=.1, hspace=0)

for i, (k,v) in enumerate(x.items()):
  print('Behandler %i af %i...' % (i+1, n))
  title_str = str(k).split('.')[0]
  f, t, Zxx = signal.stft(v,fs=fs, 
                          window='hanning',
                          nperseg=2**10, #2**12,
                          nfft=2**10)
  idx = (100 < f) & (f < 10000)
  Zxx_dB = 10*np.log10(np.abs(Zxx[idx, :])+1e-10)
  im = axs[i].pcolormesh(t,np.log10(f[idx]), Zxx_dB,
                   cmap = plt.get_cmap('viridis'),
                             vmin=-20, vmax=np.max(Zxx_dB))
  
  title_str = str(k).split('.')[0]
  axs[i].text(15,3.7,title_str, color='r',
             fontsize=stoerrelse*15, 
              horizontalalignment='center',verticalalignment='center')
  axs[i].axis('off')
  axs[i].set_xlim([0,30])

for j in range(i+1, k1*k2):
  axs[j].axis('off')
  
print('Samler alle spektrogrammerne i en figur... (kan tage lidt tid)')
plt.show()  


